In [7]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/인프런 - 파이썬 머신러닝 완벽가이드/Files/추천 - 영화 추천/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/인프런 - 파이썬 머신러닝 완벽가이드/Files/추천 - 영화 추천/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]

In [9]:
merge_df = pd.merge(ratings, movies, on='movieId')
merge_df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [41]:
user_df = merge_df.pivot_table(values='rating', index = 'userId', columns='title')
user_df.head(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
user_df.shape

(610, 9719)

In [64]:
R = user_df.values
K = 50
num_users, num_items = R.shape

np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users, K))
Q = np.random.normal(scale = 1./K, size = (num_items, K))

In [65]:
print(P.shape, Q.shape)

(610, 50) (9719, 50)


In [66]:
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

In [85]:
import sklearn
from sklearn.metrics import mean_squared_error

def get_rmse(P, Q, R, non_zeros):
  a = [non_zero[0] for non_zero in non_zeros]
  b = [non_zero[1] for non_zero in non_zeros]
  P_Q = np.dot(P, Q.T)
  pred_df = P_Q[a,b]
  R_true = R[a,b]
  mse = mean_squared_error(R_true, pred_df)
  rmse = np.sqrt(mse)

  return rmse

In [87]:
steps=200
learning_rate=0.01
r_lambda=0.01
for step in range(steps):
  for i,j,r in non_zeros:
    eij = r - np.dot(P[i,:], Q[j,:].T)
    P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
    Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])
  
  rmse = get_rmse(P, Q, R, non_zeros)
  if (step % 20) == 0:
    print("step ", step, ":", rmse)

step  0 : 0.23068944474985006
step  20 : 0.197011030799766
step  40 : 0.17912477390263123
step  60 : 0.16806822802556187
step  80 : 0.16055710758610442
step  100 : 0.15512273722053724
step  120 : 0.15100681462690715
step  140 : 0.14777804606749576
step  160 : 0.14517358368761343
step  180 : 0.1430245637522027


In [88]:
P_Q_T = np.dot(P, Q.T)

In [94]:
reco_pred_df = pd.DataFrame(data = P_Q_T, index = user_df.index, columns=user_df.columns)
reco_pred_df.head(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.034264,4.037428,3.523431,4.446798,3.933594,1.247043,3.597195,2.325715,4.977662,3.948134,1.596781,3.857809,4.773255,3.799469,3.873156,1.562920,2.862383,2.667367,3.176516,3.242834,2.810493,3.276546,2.379872,3.149907,4.104748,1.816183,3.238186,3.428108,4.411033,3.776053,3.175707,3.516553,3.836730,3.923575,5.001503,3.982475,3.505149,3.921351,3.183716,4.374963,...,3.527036,3.206238,2.046881,3.265397,1.788937,3.819789,4.231806,3.679396,2.991921,4.592312,4.650108,4.260246,3.069518,3.209911,0.984276,1.803717,4.633068,2.255096,4.126693,0.319155,4.777363,1.891552,3.526981,2.825553,3.784974,2.691472,2.362562,2.362556,4.748893,4.213489,1.399670,4.184313,3.648467,2.693308,2.774795,3.423462,3.227012,2.147506,4.002098,0.853339
2,3.168901,3.593926,3.254540,4.091711,4.226997,1.266740,4.291911,1.854230,3.305054,3.576889,1.478781,3.544974,3.742429,3.098580,3.505194,0.981902,3.940935,2.136939,3.331895,2.440348,3.425930,2.593611,1.804453,3.294698,2.593492,1.404923,2.986108,3.137233,4.196390,2.705052,2.863301,3.088023,3.720464,3.736892,3.218432,3.371798,2.906246,3.283373,2.783265,3.469831,...,3.265404,2.659010,2.036302,3.373879,1.726811,3.259246,3.654627,3.122707,2.421393,3.859002,4.193904,3.921096,3.128121,2.808558,0.819063,1.367296,3.951101,2.685175,3.844542,0.290518,3.057495,1.546812,3.073788,2.764289,3.958583,2.478346,2.196787,1.748367,3.880901,3.729411,0.963849,3.507855,3.334238,2.653105,2.392192,4.211807,2.848000,1.604100,4.113545,0.707640
3,2.247627,1.543231,1.353410,2.119162,2.155471,0.742773,1.835980,0.851976,2.973763,2.424758,0.862516,1.678034,2.609245,1.846817,2.153890,0.678192,1.028296,1.469166,0.645183,1.505022,1.688458,0.971680,1.166645,1.271555,1.713386,0.572507,1.704368,1.863661,2.249331,2.262897,1.942056,1.686848,2.208933,1.984972,2.403180,1.954879,2.451631,2.862948,1.709818,1.710871,...,1.682410,2.174489,1.365234,2.276558,1.585298,2.177237,2.5

In [115]:
def get_unseen_movie(Data, userID):
  user_movie = Data.loc[userID,:]
  already_seen  = user_movie[user_movie > 0].index.tolist()
  movies_list = Data.columns.tolist()
  unseen_list = [ movie for movie in movies_list if movie not in already_seen]

  return unseen_list

In [116]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [123]:
recommend_movies = recomm_movie_by_userid(reco_pred_df, 9, unseen_list=unseen_list, top_n=10)
print(recommend_movies)

title
Rear Window (1954)                                       5.635972
South Park: Bigger, Longer and Uncut (1999)              5.362404
Rounders (1998)                                          5.304953
Blade Runner (1982)                                      5.193145
Gattaca (1997)                                           5.170704
Roger & Me (1989)                                        5.159782
Ben-Hur (1959)                                           5.124627
Rosencrantz and Guildenstern Are Dead (1990)             5.061523
Big Lebowski, The (1998)                                 5.033498
Star Wars: Episode V - The Empire Strikes Back (1980)    5.003166
Name: 9, dtype: float64


In [125]:
unseen_list = get_unseen_movie(user_df, 9)

recommend_movies = recomm_movie_by_userid(reco_pred_df, 9, unseen_list=unseen_list, top_n=10)

reco_movie_df = pd.DataFrame(data = recommend_movies.values ,index = recommend_movies.index ,columns=['recommend ranking point'])
reco_movie_df

,recommend ranking point
title,
Rear Window (1954),5.635972
"South Park: Bigger, Longer and Uncut (1999)",5.362404
Rounders (1998),5.304953
Blade Runner (1982),5.193145
Gattaca (1997),5.170704
Roger & Me (1989),5.159782
Ben-Hur (1959),5.124627
Rosencrantz and Guildenstern Are Dead (1990),5.061523
"Big Lebowski, The (1998)",5.033498
